# Publish the Model

In this notebook, we aim to deploy the registered model from the ML Pipeline task.

In the previous step, we manage to create an ML pipeline and run it from the DevOps Build Pipeline. To make that possible, we leveraged Service Principle Identities to log in into our Azure Account from the application to access the ML Workspace. In this notebook, we don't follow this but the Python version of this notebook (located at Score/deploy_model.py) uses the same log-in mechanism, as we need to run the  from the Release Pipeline.

To register the model we follow these steps:
1. Log in to Azure and get a hold on the Workspace
1. Get the latest registered model from the Model Store
1. Create a container image from the model
1. Deploy the image as a Web Service on Azure Container Instance
1. Test the deployed model behind the web service
1. Create a Kubernetes cluster
1. Deploy the image as a Web Service into the Kubernetes
1. Test whether the model is working as expected behind the WebService within the Kubernetes cluster
1. Convert this notebook into Python and register it into the Release Pipeline

**Important Note:**:
Similar to other notebooks, here are the requirements to run this notebook:

In order to practice all parts of the following Notebook, you first need to get a free Azure credit. If you don't have it, you can simply obtain it through this link: https://azure.microsoft.com/en-us/free/

You can run this notebook on your local latop, Azure Notebooks (notebooks.azure.com) or Notebook VMs:
- Local Laptop - the following packages has to be installed:
    - Azureml-SDK - with notebook,widget extensions
    - tensorflow==1.13
- Azure Notebooks:
    - This is a free notebook, all of the packages for an ML experiment is installed
- AzureML Notebook:
    - This is a premium notebook that you can choose the VM type. Avoid using this feature for the workshop as you may burn your credit before the end or the workshop.

Once you chose the execution environment, you need to create an Azure Machine Learning Service. Follow this instruction to build one:

The following text is copied from: https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-1st-experiment-sdk-setup#create-a-workspace

An Azure Machine Learning workspace is a foundational resource in the cloud that you use to experiment, train, and deploy machine learning models. It ties your Azure subscription and resource group to an easily consumed object in the service.

You create a workspace via the Azure portal, a web-based console for managing your Azure resources.

1. Sign in to the Azure portal by using the credentials for the Azure subscription you use.
1. In the upper-left corner of Azure portal, select + Create a resource.
1. Create a new resource
1. Use the search bar to find Machine Learning service workspace.
1. Select Machine Learning service workspace.
1. In the Machine Learning service workspace pane, select Create to begin.
1. Provide the following information to configure your new workspace:
    - **Field	Description**
    - **Workspace name**: type in **FirstExample**.
    - **Subscription**: Select the Azure subscription that you want to use. (Your free credit)
    - **Resource group**: type in **MLOpsWorkshop**
    - **Location**: type in **westus2**
1. After you are finished configuring the workspace, select Create.
When the process is finished, a deployment success message appears.
1. To view the new workspace, select Go to resource.


You can explore the resource from two view:
1. https://portal.azure.com (you can access all resources including Azure ML)
1. https://ml.azure.com (recently released - still in preview and dedicated to Azure ML)



In [1]:
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.compute import AksCompute
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import ContainerImage

SDK version: 1.9.0


In [2]:
# Your subscription ID will be different replace the stirng with yours
subscription_id = "b198933e-f055-498f-958d-0726ab11eddb"
resource_group = "MLOps_Template"
workspace_name = "MLOps_template_ML"
workspace_region = "West US 2"

In [3]:
# import the Workspace class and check the azureml SDK version
# exist_ok checks if workspace exists or not.

from azureml.core import Workspace

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

In [4]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: MLOps_template_ML
Azure region: westus2
Subscription id: b198933e-f055-498f-958d-0726ab11eddb
Resource group: MLOps_Template


### 1. Get the registered model registered in the last session.

You need to make sure the model_name is set property. In order to find out the model_name you registered in the previous session, go to portal.azure.com, click on your Workspace and the from the Asset panel, click on the Models. If you don't see any model registered here, you need to first run the previous lab located at (https://github.com/classicboyir/ml-pipelines/blob/master/MLPipeline_MNIST.ipynb). Clone this repo and follow the instructions.

In [5]:
from azureml.core.model import Model

model_name = "tf_mnist_pipeline_devops.model"
model_root = Model(ws, name=model_name)

### 2. Provide the dependencies required at the inference time.

This code block generates the conda dependency file to load a TF model. You can find the script that is in charge of the inference task. You can check it at here **Score/score.py**.

In [41]:
print(os.getcwd())
#os.chdir('./score')
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

conda = None
myenv = Environment(name="myenv")
myenv.python.conda_dependencies = CondaDependencies.create(conda_packages=None,python_version='3.6.2', pip_packages=[
     'azureml-defaults',
     'inference-schema',
     'numpy',
     "azureml-monitoring",
     'tensorflow==1.13.1'
     ])


C:\Users\niloofar.nayebi\PROJECT\10-AltaGas-MLops\Avanade\build-release-ci-cd\score


### 3. Containerize the model

In this task, it builds a docker image containing the model, the dependency file and also the score.py file. The image is registered at the Images section within your Workspace. You can explore it through portal.azure.com.


### 6. (Non-free subscription) Creating a Kubernetes Cluster

In this step, we create one Kubernetes cluster to host the dockerized image and provide it through a web service.

**Important Note**:
This section requires you to upgrade your subscription from free to pay-as-you-go. Creation of Kubernetes cluster under free subscription is not supported as of now.


In [20]:
%%time

from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

aks_name = 'aks-deployment'

try:
    kubeconfig = ComputeTarget(workspace=ws, name=aks_name)
    aks_target = kubeconfig
    print('Found existing AKS service')
except Exception:
    prov_config = AksCompute.provisioning_configuration()

    print('Creating a new AKS service...')
    aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

    aks_target.wait_for_completion(show_output = True)
    print(aks_target.provisioning_state)
    print(aks_target.provisioning_errors)

# use get_status() to get a detailed status for the current cluster. 
print(aks_target.get_status())

Found existing AKS service
Succeeded
Wall time: 644 ms


In [12]:
kubeconfig.get_status()

'Succeeded'

### 7. (Non-free subscription) Registering the dockerized image as a Webservice on the Kubernetes cluster

First, we provide the configuration for the web service. We enable app insights to be able to log and monitor the activities of our mode. We can also turn on the auto scaling feature to scale the container as needed.

In [42]:
from azureml.core.model import InferenceConfig
inf_config = InferenceConfig(entry_script='score.py', environment=myenv)



In [43]:
%%time

# In this code block, we register the image under a webservice in Kubernetes cluster:

aks_service_name ='mnist-webapp-service-monitored'

aks_config = AksWebservice.deploy_configuration(autoscale_enabled=True, 
                                                       autoscale_min_replicas=1,                                               cpu_cores=2, memory_gb=1, 
                                                       scoring_timeout_ms=5000, 
                                                       replica_max_concurrent_requests=2, 
                                                       max_request_wait_time=5000, 
                                                       enable_app_insights=True,
                                                       collect_model_data=True)

try:
    aks_service = Webservice(name = aks_service_name, workspace = ws)
    print('Found the webservice, deleting the service to add a new one')
    aks_service.delete()
    print('Old webservice is deleted')
except Exception:
    print("This webservice doesn't exist")
finally:
    print('Deploying the new web service')
    # aks_service = Webservice.deploy_from_image(workspace = ws, 
    #                                        name = aks_service_name,
    #                                        image = image,
    #                                        deployment_config = aks_config,
    #                                        deployment_target = aks_target)
    aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models = [model_root],
                           inference_config=inf_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                           overwrite=True)


    aks_service.wait_for_deployment(show_output = True)
    print('This webservice is deployed')

Found the webservice, deleting the service to add a new one
Old webservice is deleted
Deploying the new web service
Running...
Succeeded
AKS service creation operation finished, operation "Succeeded"
This webservice is deployed
Wall time: 47.1 s


In [44]:
aks_service.get_logs()

'2020-08-11T21:28:49,361941101+00:00 - gunicorn/run \n2020-08-11T21:28:49,364575907+00:00 - nginx/run \n2020-08-11T21:28:49,363784505+00:00 - rsyslog/run \n2020-08-11T21:28:49,365181509+00:00 - iot-server/run \n/usr/sbin/nginx: /azureml-envs/azureml_3d2424877914924cfc59568bb0efc769/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_3d2424877914924cfc59568bb0efc769/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_3d2424877914924cfc59568bb0efc769/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_3d2424877914924cfc59568bb0efc769/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_3d2424877914924cfc59568bb0efc769/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

You can always access the Webservice URL and the authentication keys using the following commands:

In [23]:
aks_service.scoring_uri

'http://52.229.21.121:80/api/v1/service/mnist-webapp-service-monitored/score'

In [24]:
aks_service.get_keys()

('R8pMM9NYdysSKnwdyE87o18caaYAYQrd', 'IUehsMVUK76ySxKD2dIHhvzu1X4qrT4Q')